In [3]:
import warnings

import pandas as pd
import numpy as np
import pickle
import operator
import re
import gc
import gensim

# from tw_dataset.dbmodels import *
import json

In [4]:
with open('texto_tweets_seguidores_cands') as f:
    tweets = json.load(f)

In [5]:
def preprocess(doc):
    pre_doc = doc
        
    # remover URLs
    pre_doc = re.sub(
        r"https?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",
        " ", pre_doc)
    
    # minúsculas
    pre_doc = pre_doc.lower()

    # volar acentos
#     pre_doc = gensim.utils.deaccent(pre_doc)

    # remove bullshit
    pre_doc = re.sub(r"\'|\"|\\|…|\/|\-|\||\(|\)|\.|\,|\!|\?|\:|\;|“|”|’|—", " ", pre_doc)
    
    # contraer vocales
    for v in 'aeiou':
        pre_doc = re.sub(r"[%s]+" % v, v, pre_doc)    

    # volar menciones
    pre_doc = re.sub(r"\@\w+"," ", pre_doc)
    
    # volar hashtags
    pre_doc = re.sub(r"\B(\#[a-zA-Z]+\b)(?!;)"," ", pre_doc)
    
    # normalizar espacio en blanco
    pre_doc = re.sub(r"\s+", " ", pre_doc)
    pre_doc = re.sub(r"(^\s)|(\s$)", "", pre_doc)
    
    
    return pre_doc

In [6]:
tws = tweets['HectorBaldassi']

In [7]:
t = tws[1]
print(t)

RT @carobarbero0830: @actcargentina recorrimos 23000 km con el #caminoalos80 #orgulloTC https://t.co/EXi5rFy9IG


In [8]:
def hashtags(t):
    return re.findall(r"\B(\#[a-zA-Z]+\b)(?!;)", t)

In [9]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.data import load
from nltk.stem import SnowballStemmer
from string import punctuation

spanish_tokenizer = load('tokenizers/punkt/spanish.pickle')

# stopwords en español
spanish_stopwords = stopwords.words('spanish')

# spanish stemmer
stemmer = SnowballStemmer('spanish')

# punctuation to remove
non_words = list(punctuation)

# we add spanish punctuation
non_words.extend(['¿', '¡'])
non_words.extend(map(str, range(10)))

stemmer = SnowballStemmer('spanish')

def trystem(t):
    try:
        t = stemmer.stem(t)
    except Exception:
        pass
    return t

def tokenize(text, stem=False, remove_stopwords=False):
    text = text.lower()
    result = []
    
    for sentence in spanish_tokenizer.tokenize(text):
        # remover puntuación
#         text = ''.join([c for c in sentence if c not in non_words])
        
        # tokenize
        tokens = word_tokenize(text)

        if remove_stopwords:
            tokens = [t for t in tokens if t not in spanish_stopwords]

        # tokens de al menos 2 letras
        tokens = [t for t in tokens if len(t) > 1]
            
        # stem
        if stem:
            tokens = [trystem(t) for t in tokens]
        
        result += tokens
        
    return result

In [11]:
class get_docs(object):
    def __init__(self, corpus):
        self.corpus = corpus

    def __iter__(self):
        for doc in self.corpus:
            tokens = tokenize(preprocess(doc), remove_stopwords=True, stem=False)
            yield tokens

In [25]:
tws = tweets['HectorBaldassi']

In [26]:
len(tws)

1223

In [27]:
token_tws = list(get_docs(tws))

In [28]:
from collections import defaultdict

In [30]:
counts = defaultdict(int)

for tokens in token_tws:
    for t in tokens:
        counts[t] += 1

# dictionary.filter_extremes(no_below=100, no_above=0.3, keep_n=None)
for w, c in sorted(counts.items(), key=lambda x:-x[1])[:150]:
    print("%s: %d" % (w, c))

rt: 905
más: 59
si: 56
gracias: 55
pepsico: 47
hoy: 35
está: 35
breve: 35
seguirme: 35
macri: 34
follow: 34
devuelvo: 34
vía: 33
https: 29
justicia: 28
argentina: 26
vido: 26
años: 24
silencio: 24
paga: 22
trabajadores: 21
nuevos: 19
hacer: 19
000: 19
gente: 17
ahora: 17
18: 17
hace: 17
seguidores: 17
país: 17
gils: 17
gran: 17
día: 16
dos: 16
vez: 16
vamos: 16
20: 16
ser: 16
solo: 16
santa: 16
córdoba: 16
3a: 16
unfollowers: 15
juez: 15
trabajo: 15
así: 15
ley: 15
va: 15
argentinos: 15
millones: 15
últimas: 14
james: 14
cristina: 14
cada: 14
cfk: 14
también: 14
gobierno: 13
seguir: 13
obras: 13
agosto: 13
http…: 13
jueces: 13
marihuana: 13
qué: 13
mejor: 13
24h: 12
plata: 12
ex: 12
10: 12
15: 12
viale910: 12
acá: 12
https…: 12
vos: 12
nuevo: 12
nueva: 12
junto: 12
policía: 12
cruz: 12
puede: 12
horas: 11
represión: 11
massa: 11
cómo: 11
siempre: 11
kirchner: 11
12: 11
carbó: 11
incautamos: 11
22: 11
marcha: 11
izquierda: 11
detuvimos: 11
presidente: 11
tras: 11
mil: 11
corrupción: 11


# Hashtags

In [21]:
tws = tweets["MartinLlaryora"]

In [22]:
hashtags_cand = [hashtags(preprocess(t)) for t in tws]

In [23]:
counts = defaultdict(int)

for hts in hashtags_cand:
    for ht in hts:
        counts[ht] += 1

# dictionary.filter_extremes(no_below=100, no_above=0.3, keep_n=None)
for w, c in sorted(counts.items(), key=lambda x:-x[1])[:40]:
    print("%s: %d" % (w, c))

#belgrano: 9
#cba: 9
#pepsico: 7
#caba: 6
#macrireprimepepsico: 5
#c: 5
#macri: 5
#pymes: 4
#producci: 4
#schiaretti: 4
#provincia: 4
#instituto: 4
#radicales: 4
#massismo: 4
#cr: 4
#campa: 4
#nodesalojenpepsico: 4
#recursos: 4
#cortocircuito: 3
#pais: 3
#jonathanramis: 3
#cambiemos: 3
#arg: 3
#pretemporadabelgrano: 3
#impuestos: 3
#ajuste: 3
#naci: 2
#cgt: 2
#empresas: 2
#econom: 2
#legislativas: 2
#devido: 2
#decisionempresa: 2
#gobiernoupc: 2
#criticas: 2
#hoy: 2
#apoyolaluchadepepsico: 2
#consultapopular: 2
#puertapuerta: 2
#politica: 2
